In [1]:
import urllib
import random
import re
import nltk
import string
nltk.download('punkt')
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\PCZ\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\PCZ\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\PCZ\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import TruncatedSVD
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC
from xgboost import XGBClassifier
import pandas as pd

In [3]:
auth1_url1 = "https://www.gutenberg.org/files/786/786-0.txt"
auth1_url2 = "https://www.gutenberg.org/files/564/564-0.txt"
auth1_url3 = "https://www.gutenberg.org/files/675/675-0.txt"
auth1_url4 = "https://www.gutenberg.org/files/98/98-0.txt"
auth1_url5 = "https://www.gutenberg.org/files/650/650-0.txt"

In [4]:
auth2_url1 = "https://www.gutenberg.org/files/141/141-0.txt"
auth2_url2 = "https://www.gutenberg.org/cache/epub/21839/pg21839.txt"
auth2_url3 = "https://www.gutenberg.org/cache/epub/42671/pg42671.txt"
auth2_url4 = "https://www.gutenberg.org/cache/epub/105/pg105.txt"
auth2_url5 = "https://www.gutenberg.org/files/121/121-0.txt"

In [5]:
auth3_url1 = "https://www.gutenberg.org/files/108/108-0.txt"
auth3_url2 = "https://www.gutenberg.org/files/834/834-0.txt"
auth3_url3 = "https://www.gutenberg.org/files/903/903-0.txt"
auth3_url4 = "https://www.gutenberg.org/files/48320/48320-0.txt"
auth3_url5 = "https://www.gutenberg.org/files/54109/54109-0.txt"

In [6]:
auth4_url1 = "https://www.gutenberg.org/files/28289/28289-0.txt"
auth4_url2 = "https://www.gutenberg.org/cache/epub/17780/pg17780.txt"
auth4_url3 = "https://www.gutenberg.org/files/550/550-0.txt"
auth4_url4 = "https://www.gutenberg.org/cache/epub/47025/pg47025.txt"
auth4_url5 = "https://www.gutenberg.org/files/10762/10762.txt"

In [7]:
auth5_url1 = "https://www.gutenberg.org/cache/epub/60278/pg60278.txt"
auth5_url2 = "https://www.gutenberg.org/cache/epub/66837/pg66837.txt"
auth5_url3 = "https://www.gutenberg.org/files/60327/60327-0.txt"
auth5_url4 = "https://www.gutenberg.org/files/3474/3474-0.txt"
auth5_url5 = "https://www.gutenberg.org/cache/epub/27180/pg27180.txt"

In [29]:
def extract_fragments(url, L):
    #Read the file
    file = urllib.request.urlopen(url)
    lines = []
    for line in file :
        decoded_line = line.decode("utf-8")
        lines.append(decoded_line.strip())
    
    #Change the letters to lower case
    raw_docs = [doc.lower() for doc in lines]
    while ("" in raw_docs) :
        raw_docs.remove("")
    
    #Word tokenization
    tokenized_doc = [word_tokenize(doc) for doc in raw_docs]
    
    #Remove the punctuations
    regex = re.compile('[%s]'% re.escape(string.punctuation))
    tokenized_docs_no_punc = []
    for review in tokenized_doc :
        new_review = []
        for token in review :
            new_token = regex.sub(u'', token)
            if not new_token == u'' :
                new_review.append(new_token)
        tokenized_docs_no_punc.append(new_review)
    
    puncs = ["’", "‘", "“", "”"]
    for punc in puncs :
        for line in tokenized_docs_no_punc :
            while(punc in line) :
                line.remove(punc)
    
    #Remove the stop words
    tokenized_doc_no_sw = []
    for doc in tokenized_docs_no_punc :
        new_term_vector = []
        for word in doc :
            if not word in stopwords.words('english') :
                new_term_vector.append(word)
        tokenized_doc_no_sw.append(new_term_vector)
    
    lists = []
    for line in tokenized_doc_no_sw:
        lines = []
        for word in line:
            lines.append("".join(u for u in word if u not in ("—")))
        lists.append(lines)
    
    #Use the lemmantization to get the root words
    wordnet = WordNetLemmatizer()
    pre_docs = []
    for doc in lists :
        final_doc = []
        for word in doc :
            final_doc.append(wordnet.lemmatize(word))
        pre_docs.append(final_doc)
    
    #Extracting L number of fragments in size of 200
    final_list = []
    i = 199
    while len(final_list) < 200 :
            line = pre_docs[i+1]
            while len(line) <= L:
                i = i + 2
                for word in pre_docs[i]:
                    line.append(word)
            final_list.append(' '.join(line[0:L]))
    return final_list

In [30]:
L = 50

In [31]:
auth1_list1 = extract_fragments(auth1_url1, L)
auth1_list2 = extract_fragments(auth1_url2, L)
auth1_list3 = extract_fragments(auth1_url3, L)
auth1_list4 = extract_fragments(auth1_url4, L)
auth1_list5 = extract_fragments(auth1_url5, L)

In [32]:
list1 = [*auth1_list1, *auth1_list2, *auth1_list3, *auth1_list4, *auth1_list5]

In [33]:
auth2_list1 = extract_fragments(auth2_url1, L)
auth2_list2 = extract_fragments(auth2_url2, L)
auth2_list3 = extract_fragments(auth2_url3, L)
auth2_list4 = extract_fragments(auth2_url4, L)
auth2_list5 = extract_fragments(auth2_url5, L)

In [34]:
list2 = [*auth2_list1, *auth2_list2, *auth2_list3, *auth2_list4, *auth2_list5]

In [35]:
auth3_list1 = extract_fragments(auth3_url1, L)
auth3_list2 = extract_fragments(auth3_url2, L)
auth3_list3 = extract_fragments(auth3_url3, L)
auth3_list4 = extract_fragments(auth3_url4, L)
auth3_list5 = extract_fragments(auth3_url5, L)

In [36]:
list3 = [*auth3_list1, *auth3_list2, *auth3_list3, *auth3_list4, *auth3_list5]

In [37]:
auth4_list1 = extract_fragments(auth4_url1, L)
auth4_list2 = extract_fragments(auth4_url2, L)
auth4_list3 = extract_fragments(auth4_url3, L)
auth4_list4 = extract_fragments(auth4_url4, L)
auth4_list5 = extract_fragments(auth4_url5, L)

In [38]:
list4 = [*auth4_list1, *auth4_list2, *auth4_list3, *auth4_list4, *auth4_list5]

In [39]:
auth5_list1 = extract_fragments(auth5_url1, L)
auth5_list2 = extract_fragments(auth5_url2, L)
auth5_list3 = extract_fragments(auth5_url3, L)
auth5_list4 = extract_fragments(auth5_url4, L)
auth5_list5 = extract_fragments(auth5_url5, L)

In [40]:
list5 = [*auth5_list1, *auth5_list2, *auth5_list3, *auth5_list4, *auth5_list5]

In [71]:
x = []
x = [*list1, *list2, *list3, *list4, *list5]

In [72]:
len(x), x[0]

(5000,
 'beginning sunbeam bitzer corner row side row advance caught end receive deeper lustrous colour sun shone ray appeared draw little colour ever possessed lash bringing immediate contrast something might mere continuation sandy freckle forehead looked though cut would bleed white quadruped graminivorous forty teeth namely twentyfour grinder country shed hoof')

In [167]:
authors = ['Charles Dickens', 'Jane Austen', 'Sir Arthur Conan Doyle', 'George Eliot', 'Hugh Walpole']
target = []
for author in authors :
    for i in range(0, 1000):
        target.append(author)
y = LabelEncoder().fit_transform(target)

In [168]:
len(y), set(y)

(5000, {0, 1, 2, 3, 4})

In [169]:
data_df = pd.DataFrame({
    'Extracted Fragments' : [],
    'Author' : []
})

In [170]:
for i in range (0, len(x)) :
    data_df = data_df.append({
        'Extracted Fragments' : x[i],
        'Author' : target[i],
    }, ignore_index = True
    )

In [171]:
data_df

,Extracted Fragments,Author
0,beginning sunbeam bitzer corner row side row a...,Charles Dickens
1,iron age known mark mouth thus much bitzer gir...,Charles Dickens
2,continue fistic phraseology genius coming scra...,Charles Dickens
3,ventured answer paper room would paint must pa...,Charles Dickens
4,general conviction time sir always feeble stra...,Charles Dickens
...,...,...
4995,robin rate made meaning clear wish regard comp...,Hugh Walpole
4996,omitted mine came back stranger ready anything...,Hugh Walpole
4997,see please shall course disguise position worl...,Hugh Walpole
4998,end would father said show stiff back white mo...,Hugh Walpole


In [172]:
data_df.to_csv('Dataset_L50.csv')

## BOW

Logistic Regression

In [91]:
bow = CountVectorizer()
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)
x_train_bow = bow.fit_transform(x_train)

In [84]:
lr_par = {
    'penalty' : ['l2'],
    'C' : [0.1, 1.0, 10.0],
    'solver' : ['newton-cg', 'lbfgs', 'sag', 'saga'],
    'max_iter' : [100, 200, 300]
}

In [85]:
LR = LogisticRegression()
lr_grid = GridSearchCV(LR, lr_par, refit = True, n_jobs=-1, cv=5)
lr_grid.fit(x_train_bow, y_train)
opt_par = lr_grid.best_params_
opt_par

{'C': 10.0, 'max_iter': 100, 'penalty': 'l2', 'solver': 'newton-cg'}

In [86]:
LR = LogisticRegression(C = 10.0, max_iter = 100, penalty = 'l2', solver = 'newton-cg')

In [87]:
lr_train_acc = []
lr_test_acc = []
for i in range (0, 20):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = i+1)
    bow = CountVectorizer()
    x_train_bow = bow.fit_transform(x_train)
    x_test_bow = bow.transform(x_test)
    LR.fit(x_train_bow, y_train)
    y_pred = LR.predict(x_test_bow)
    y_train_pred = LR.predict(x_train_bow)
    lr_train_acc.append(metrics.accuracy_score(y_train, y_train_pred))
    lr_test_acc.append(metrics.accuracy_score(y_test, y_pred))

In [88]:
lr_mean_train_acc_bow = sum(lr_train_acc)/len(lr_train_acc)
lr_mean_test_acc_bow = sum(lr_test_acc)/len(lr_test_acc)

In [89]:
lr_mean_train_acc_bow, lr_mean_test_acc_bow

(1.0, 0.9194000000000001)

Support Vector Machines

In [94]:
bow = CountVectorizer()
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
x_train_bow = bow.fit_transform(x_train)

In [92]:
svm_par = [
   { 'C' : [0.1, 1.0, 10.0],
    'kernel' : ['linear']
   },
   { 'C' : [0.1, 1.0, 10.0],
    'kernel' : ['poly'],
    'degree' : [2, 3, 4],
    'gamma' : ['scale', 'auto']
   },
   { 'C' : [0.1, 1.0, 10.0],
    'kernel' : ['rbf', 'sigmoid'],
    'gamma' : ['scale', 'auto']
    }
]

In [93]:
SVM = SVC()
svm_grid = GridSearchCV(SVM, svm_par, refit = True, n_jobs=-1, cv=5)
svm_grid.fit(x_train_bow, y_train)
opt_par = svm_grid.best_params_
opt_par

{'C': 0.1, 'kernel': 'linear'}

In [95]:
SVM = SVC( C = 0.1, kernel = 'linear' )

In [96]:
svm_train_acc = []
svm_test_acc = []
for i in range (0, 20):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = i+1)
    bow = CountVectorizer()
    x_train_bow = bow.fit_transform(x_train)
    x_test_bow = bow.transform(x_test)
    SVM.fit(x_train_bow, y_train)
    y_pred = SVM.predict(x_test_bow)
    y_train_pred = SVM.predict(x_train_bow)
    svm_train_acc.append(metrics.accuracy_score(y_train, y_train_pred))
    svm_test_acc.append(metrics.accuracy_score(y_test, y_pred))

In [97]:
svm_mean_train_acc_bow = sum(svm_train_acc)/len(svm_train_acc)
svm_mean_test_acc_bow = sum(svm_test_acc)/len(svm_test_acc)

In [98]:
svm_mean_train_acc_bow, svm_mean_test_acc_bow

(1.0, 0.9015500000000001)

Random Forest

In [99]:
bow = CountVectorizer()
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
x_train_bow = bow.fit_transform(x_train)

In [100]:
rf_par = {
    'n_estimators' : [50, 100, 200],
    'criterion' : ['gini', 'entropy'],
    'max_depth' : [8, 16, 32],
    'max_features' : ['sqrt', 'log2']
}

In [101]:
RF = RandomForestClassifier()
rf_grid = GridSearchCV(RF, rf_par, refit = True, n_jobs=-1, cv=5)
rf_grid.fit(x_train_bow, y_train)
opt_par = rf_grid.best_params_
opt_par

{'criterion': 'gini',
 'max_depth': 32,
 'max_features': 'log2',
 'n_estimators': 200}

In [102]:
RF = RandomForestClassifier(criterion = 'gini', max_depth = 32, max_features = 'log2', n_estimators = 200 )

In [103]:
rf_train_acc = []
rf_test_acc = []
for i in range (0, 20):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = i+1)
    bow = CountVectorizer()
    x_train_bow = bow.fit_transform(x_train)
    x_test_bow = bow.transform(x_test)
    RF.fit(x_train_bow, y_train)
    y_pred = RF.predict(x_test_bow)
    y_train_pred = RF.predict(x_train_bow)
    rf_train_acc.append(metrics.accuracy_score(y_train, y_train_pred))
    rf_test_acc.append(metrics.accuracy_score(y_test, y_pred))

In [104]:
rf_mean_train_acc_bow = sum(rf_train_acc)/len(rf_train_acc)
rf_mean_test_acc_bow = sum(rf_test_acc)/len(rf_test_acc)

In [105]:
rf_mean_train_acc_bow, rf_mean_test_acc_bow

(0.9998499999999998, 0.8686999999999998)

Naive Bays 

In [108]:
NB = MultinomialNB()

In [109]:
nb_train_acc = []
nb_test_acc = []
for i in range (0, 20):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = i+1)
    bow = CountVectorizer()
    x_train_bow = bow.fit_transform(x_train)
    x_test_bow = bow.transform(x_test)
    NB.fit(x_train_bow, y_train)
    y_pred = NB.predict(x_test_bow)
    y_train_pred = NB.predict(x_train_bow)
    nb_train_acc.append(metrics.accuracy_score(y_train, y_train_pred))
    nb_test_acc.append(metrics.accuracy_score(y_test, y_pred))

In [110]:
nb_mean_train_acc_bow = sum(nb_train_acc)/len(nb_train_acc)
nb_mean_test_acc_bow = sum(nb_test_acc)/len(nb_test_acc)

In [111]:
nb_mean_train_acc_bow, nb_mean_test_acc_bow

(0.991, 0.9328000000000001)

XGBoost

In [112]:
bow = CountVectorizer()
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
x_train_bow = bow.fit_transform(x_train)

In [113]:
xgb_par = {
    'n_estimators' : [20, 50, 100],
    'use_label_encoder' : [False],
    'max_depth' : [8, 16, 32],
    'learning_rate' : [0.01, 0.1]
}

In [114]:
XGB = XGBClassifier()
xgb_grid = GridSearchCV(XGB, xgb_par, refit = True, n_jobs=-1, cv=5)
xgb_grid.fit(x_train_bow, y_train)
opt_par = xgb_grid.best_params_
opt_par

[20:58:49] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


{'learning_rate': 0.1,
 'max_depth': 8,
 'n_estimators': 100,
 'use_label_encoder': False}

In [115]:
XGB = XGBClassifier(learning_rate = 0.1, max_depth = 8, n_estimators = 100, use_label_encoder = False )

In [116]:
xgb_train_acc = []
xgb_test_acc = []
for i in range (0, 20):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = i+1)
    bow = CountVectorizer()
    x_train_bow = bow.fit_transform(x_train)
    x_test_bow = bow.transform(x_test)
    XGB.fit(x_train_bow, y_train)
    y_pred = XGB.predict(x_test_bow)
    y_train_pred = XGB.predict(x_train_bow)
    xgb_train_acc.append(metrics.accuracy_score(y_train, y_train_pred))
    xgb_test_acc.append(metrics.accuracy_score(y_test, y_pred))

[21:00:01] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[21:00:17] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[21:00:32] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[21:00:47] WARNING: C:/Users/Administrator/workspace/xgboost-win64_

In [117]:
xgb_mean_train_acc_bow = sum(xgb_train_acc)/len(xgb_train_acc)
xgb_mean_test_acc_bow = sum(xgb_test_acc)/len(xgb_test_acc)

In [118]:
xgb_mean_train_acc_bow, xgb_mean_test_acc_bow

(0.984675, 0.8283499999999998)

In [119]:
algorithms = ['Logistic Regression', 'Support Vector Machine', 'Random Forest', 'Naive Bayes', 'XGBoost']

In [120]:
bow_df = pd.DataFrame({
    'Algorithm' : [], 'Mean Train Accuracy' : [], 'Mean Test Accuracy' :[]
})

In [121]:
mean_train_acc_bow = [lr_mean_train_acc_bow, svm_mean_train_acc_bow, rf_mean_train_acc_bow,
                      nb_mean_train_acc_bow, xgb_mean_train_acc_bow]
mean_test_acc_bow = [lr_mean_test_acc_bow, svm_mean_test_acc_bow, rf_mean_test_acc_bow,
                    nb_mean_test_acc_bow, xgb_mean_test_acc_bow]

In [122]:
for i in range (0, len(algorithms)) :
    bow_df = bow_df.append({
        'Algorithm' : algorithms[i],
        'Mean Train Accuracy' : mean_train_acc_bow[i],
        'Mean Test Accuracy' : mean_test_acc_bow[i],
    }, ignore_index = True
    )

In [123]:
bow_df

,Algorithm,Mean Train Accuracy,Mean Test Accuracy
0,Logistic Regression,1.000000,0.91940
1,Support Vector Machine,1.000000,0.90155
2,Random Forest,0.999850,0.86870
3,Naive Bayes,0.991000,0.93280
4,XGBoost,0.984675,0.82835


In [124]:
bow_df.to_csv('BOW_Results.csv')

## Tf-idf

Logistic Regression

In [125]:
tf_idf = TfidfVectorizer(use_idf=True)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
x_train_tf = tf_idf.fit_transform(x_train)

In [126]:
lr_par = {
    'penalty' : ['l2'],
    'C' : [0.1, 1.0, 10.0],
    'solver' : ['newton-cg', 'lbfgs', 'sag', 'saga'],
    'max_iter' : [100, 200, 300]
}

In [127]:
LR = LogisticRegression()
lr_grid = GridSearchCV(LR, lr_par, refit = True, n_jobs=-1, cv=5)
lr_grid.fit(x_train_tf, y_train)
opt_par = lr_grid.best_params_
opt_par

{'C': 10.0, 'max_iter': 200, 'penalty': 'l2', 'solver': 'saga'}

In [128]:
LR = LogisticRegression(C = 10.0, max_iter = 200, penalty = 'l2', solver = 'saga')

In [129]:
lr_train_acc = []
lr_test_acc = []
for i in range (0, 20):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = i+1)
    tf_idf = TfidfVectorizer(use_idf=True)
    x_train_tf = tf_idf.fit_transform(x_train)
    x_test_tf = tf_idf.transform(x_test)
    LR.fit(x_train_tf, y_train)
    y_pred = LR.predict(x_test_tf)
    y_train_pred = LR.predict(x_train_tf)
    lr_train_acc.append(metrics.accuracy_score(y_train, y_train_pred))
    lr_test_acc.append(metrics.accuracy_score(y_test, y_pred))

In [130]:
lr_mean_train_acc_tf = sum(lr_train_acc)/len(lr_train_acc)
lr_mean_test_acc_tf = sum(lr_test_acc)/len(lr_test_acc)

In [131]:
lr_mean_train_acc_tf, lr_mean_test_acc_tf

(1.0, 0.9441)

Support Vector Machine

In [132]:
tf_idf = TfidfVectorizer(use_idf=True)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
x_train_tf = tf_idf.fit_transform(x_train)

In [133]:
svm_par = [
   { 'C' : [0.1, 1.0, 10.0],
    'kernel' : ['linear']
   },
   { 'C' : [0.1, 1.0, 10.0],
    'kernel' : ['poly'],
    'degree' : [2, 3, 4],
    'gamma' : ['scale', 'auto']
   },
   { 'C' : [0.1, 1.0, 10.0],
    'kernel' : ['rbf', 'sigmoid'],
    'gamma' : ['scale', 'auto']
    }
]

In [134]:
SVM = SVC()
svm_grid = GridSearchCV(SVM, svm_par, refit = True, n_jobs=-1, cv=5)
svm_grid.fit(x_train_tf, y_train)
opt_par = svm_grid.best_params_
opt_par

{'C': 1.0, 'kernel': 'linear'}

In [135]:
SVM = SVC(C = 1.0, kernel = 'linear' )

In [136]:
svm_train_acc = []
svm_test_acc = []
for i in range (0, 20):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = i+1)
    tf_idf = TfidfVectorizer(use_idf=True)
    x_train_tf = tf_idf.fit_transform(x_train)
    x_test_tf = tf_idf.transform(x_test)
    SVM.fit(x_train_tf, y_train)
    y_pred = SVM.predict(x_test_tf)
    y_train_pred = SVM.predict(x_train_tf)
    svm_train_acc.append(metrics.accuracy_score(y_train, y_train_pred))
    svm_test_acc.append(metrics.accuracy_score(y_test, y_pred))

In [137]:
svm_mean_train_acc_tf = sum(svm_train_acc)/len(svm_train_acc)
svm_mean_test_acc_tf = sum(svm_test_acc)/len(svm_test_acc)

In [138]:
svm_mean_train_acc_tf, svm_mean_test_acc_tf

(0.9992124999999998, 0.9400499999999999)

Random Forest

In [139]:
tf_idf = TfidfVectorizer(use_idf=True)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)
x_train_tf = tf_idf.fit_transform(x_train)

In [140]:
rf_par = {
    'n_estimators' : [50, 100, 200],
    'criterion' : ['gini', 'entropy'],
    'max_depth' : [8, 16, 32],
    'max_features' : ['sqrt', 'log2']
}

In [141]:
RF = RandomForestClassifier()
rf_grid = GridSearchCV(RF, rf_par, refit = True, n_jobs=-1, cv=5)
rf_grid.fit(x_train_tf, y_train)
opt_par = rf_grid.best_params_
opt_par

{'criterion': 'gini',
 'max_depth': 32,
 'max_features': 'log2',
 'n_estimators': 200}

In [142]:
RF = RandomForestClassifier(criterion = 'gini', max_depth = 32, max_features = 'log2', n_estimators = 200)

In [143]:
rf_train_acc = []
rf_test_acc = []
for i in range (0, 20):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = i+1)
    tf_idf = TfidfVectorizer(use_idf=True)
    x_train_tf = tf_idf.fit_transform(x_train)
    x_test_tf = tf_idf.transform(x_test)
    RF.fit(x_train_tf, y_train)
    y_pred = RF.predict(x_test_tf)
    y_train_pred = RF.predict(x_train_tf)
    rf_train_acc.append(metrics.accuracy_score(y_train, y_train_pred))
    rf_test_acc.append(metrics.accuracy_score(y_test, y_pred))

In [144]:
rf_mean_train_acc_tf = sum(rf_train_acc)/len(rf_train_acc)
rf_mean_test_acc_tf = sum(rf_test_acc)/len(rf_test_acc)

In [145]:
rf_mean_train_acc_tf, rf_mean_test_acc_tf

(0.98955, 0.7921)

Naive Bayes

In [150]:
NB = MultinomialNB()

In [151]:
nb_train_acc = []
nb_test_acc = []
for i in range (0, 20):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = i+1)
    tf_idf = TfidfVectorizer(use_idf=True)
    x_train_tf = tf_idf.fit_transform(x_train)
    x_test_tf = tf_idf.transform(x_test)
    NB.fit(x_train_tf, y_train)
    y_pred = NB.predict(x_test_tf)
    y_train_pred = NB.predict(x_train_tf)
    nb_train_acc.append(metrics.accuracy_score(y_train, y_train_pred))
    nb_test_acc.append(metrics.accuracy_score(y_test, y_pred))

In [152]:
nb_mean_train_acc_tf = sum(nb_train_acc)/len(nb_train_acc)
nb_mean_test_acc_tf = sum(nb_test_acc)/len(nb_test_acc)

In [153]:
nb_mean_train_acc_tf, nb_mean_test_acc_tf

(0.9757000000000001, 0.8935000000000001)

XGBoost

In [154]:
tf_idf = TfidfVectorizer(use_idf=True)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
x_train_tf = tf_idf.fit_transform(x_train)

In [155]:
xgb_par = {
    'n_estimators' : [20, 50, 100],
    'use_label_encoder' : [False],
    'max_depth' : [8, 16, 32],
    'learning_rate' : [0.01, 0.1]
}

In [156]:
XGB = XGBClassifier()
xgb_grid = GridSearchCV(XGB, xgb_par, refit = True, n_jobs=-1, cv=5)
xgb_grid.fit(x_train_tf, y_train)
opt_par = xgb_grid.best_params_
opt_par

[22:43:23] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


{'learning_rate': 0.1,
 'max_depth': 8,
 'n_estimators': 100,
 'use_label_encoder': False}

In [157]:
XGB = XGBClassifier(learning_rate = 0.1, max_depth = 8, n_estimators = 100, use_label_encoder = False)

In [158]:
xgb_train_acc = []
xgb_test_acc = []
for i in range (0, 20):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = i+1)
    tf_idf = TfidfVectorizer(use_idf=True)
    x_train_tf = tf_idf.fit_transform(x_train)
    x_test_tf = tf_idf.transform(x_test)
    XGB.fit(x_train_tf, y_train)
    y_pred = XGB.predict(x_test_tf)
    y_train_pred = XGB.predict(x_train_tf)
    xgb_train_acc.append(metrics.accuracy_score(y_train, y_train_pred))
    xgb_test_acc.append(metrics.accuracy_score(y_test, y_pred))

[22:44:28] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[22:45:11] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[22:45:54] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[22:46:37] WARNING: C:/Users/Administrator/workspace/xgboost-win64_

In [159]:
xgb_mean_train_acc_tf = sum(xgb_train_acc)/len(xgb_train_acc)
xgb_mean_test_acc_tf = sum(xgb_test_acc)/len(xgb_test_acc)

In [160]:
xgb_mean_train_acc_tf, xgb_mean_test_acc_tf

(0.9883624999999998, 0.8030000000000002)

In [161]:
algorithms = ['Logistic Regression', 'Support Vector Machine', 'Random Forest', 'Naive Bayes', 'XGBoost']

In [162]:
tf_df = pd.DataFrame({
    'Algorithm' : [], 'Mean Train Accuracy' : [], 'Mean Test Accuracy' :[]
})

In [163]:
mean_train_acc_tf = [lr_mean_train_acc_tf, svm_mean_train_acc_tf, rf_mean_train_acc_tf,
                      nb_mean_train_acc_tf, xgb_mean_train_acc_tf]
mean_test_acc_tf = [lr_mean_test_acc_tf, svm_mean_test_acc_tf, rf_mean_test_acc_tf,
                    nb_mean_test_acc_tf, xgb_mean_test_acc_tf]

In [164]:
for i in range (0, len(algorithms)) :
    tf_df = tf_df.append({
        'Algorithm' : algorithms[i],
        'Mean Train Accuracy' : mean_train_acc_tf[i],
        'Mean Test Accuracy' : mean_test_acc_tf[i],
    }, ignore_index = True
    )

In [165]:
tf_df

,Algorithm,Mean Train Accuracy,Mean Test Accuracy
0,Logistic Regression,1.000000,0.94410
1,Support Vector Machine,0.999212,0.94005
2,Random Forest,0.989550,0.79210
3,Naive Bayes,0.975700,0.89350
4,XGBoost,0.988362,0.80300


In [166]:
tf_df.to_csv('TF-IDF_Results.csv')